In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from tqdm import tqdm
import os
import ipywidgets as widgets
from IPython.display import display
import datetime

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Hyperparameters & locations
</div>

In [2]:
locs = {'waiting_folder' : '../data/waiting times'}

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Reading the data
</div>

In [3]:
waiting_times = list()
for attraction in tqdm(os.listdir(locs['waiting_folder'])):
    filename = os.path.join(locs['waiting_folder'], attraction)
    df = pd.read_csv(filename)
    if df.empty:
        continue
    df.insert(0, 'attraction', attraction.split('.')[0])
    waiting_times.append(df)

df_wait_raw = pd.concat(waiting_times, ignore_index=True)

100%|██████████| 52/52 [00:14<00:00,  3.62it/s]


<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Cleaning the data
</div>

In [4]:
df_wait = df_wait_raw.copy()
# Cleaning the actual waiting times
# Removing outliers from actuals
df_wait = df_wait[((df_wait.SACTMIN >= -1000) & (df_wait.SACTMIN < 360)) | (df_wait.SACTMIN.isnull())]
# Removing outliers from posted (attraction closed at -999)
df_wait = df_wait[(df_wait.SPOSTMIN >= -998.99) | (df_wait.SPOSTMIN.isnull())]

df_wait['date'] = pd.to_datetime(df_wait.date, format = '%m/%d/%Y')
df_wait['datetime'] = pd.to_datetime(df_wait.datetime, format = '%Y-%m-%d %H:%M:%S')

print(f"Removed {len(df_wait_raw) - len(df_wait)} rows")

# You could split the dataset into two seperate dataframes (plusjes & minnetjes :-))
df_wait_act = df_wait[~df_wait.SACTMIN.isnull()].drop('SPOSTMIN', axis = 1)
df_wait_post = df_wait[~df_wait.SPOSTMIN.isnull()].drop('SACTMIN', axis = 1)

attractions = df_wait.attraction.unique()

Removed 1318703 rows


In [5]:
# Extract "minutes from df_wait" from the datetime column
df_wait_post['minute'] = df_wait_post['datetime'].dt.hour * 60 + df_wait_post['datetime'].dt.minute

In [6]:
df_wait_post['attraction'].unique()

array(['country_bears', '7_dwarfs_train', 'pirates_of_caribbean',
       'astro_orbiter', 'laugh_floor', 'regal_carrousel',
       'big_thunder_mtn', 'spaceship_earth', 'splash_mountain',
       'hall_of_presidents', 'toy_story_mania', 'space_mountain',
       'sorcerers_of_the_mk', 'jungle_cruise', 'mad_tea_party',
       'princess_hall__cinderella_elena', 'dumbo', 'tom_land_speedway',
       'swiss_family_tree', 'magic_carpets', 'tom_sawyer_island',
       'soarin', 'peoplemover', 'philharmagic', 'it_s_a_small_world',
       'kilimanjaro_safaris', 'expedition_everest', 'town_sq_mickey',
       'rock_n_rollercoaster', 'carousel_of_progress', 'under_the_sea',
       'dinosaur', 'barnstormer', 'flight_of_passage', 'winnie_the_pooh',
       'navi_river', 'enchanted_tiki_rm', 'princess_hall__rapunzel_tiana',
       'pirate_s_adventure', 'liberty_sq_riverboat', 'peter_pan_s_flight',
       'haunted_mansion', 'alien_saucers', 'buzz_lightyear', 'slinky_dog'],
      dtype=object)

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Preprocessing - timeshift only
</div>

In [7]:
## Posted waiting times only
# from tqdm import tqdm
# groups = list()
# for (date, attraction), group in tqdm(df_wait_post.groupby(['date', 'attraction'])):
# #for date, group in tqdm(list(df_wait_post.groupby('date'))):
#     time_shifted = group.datetime - datetime.timedelta(hours = 3)
#     group['minute'] = 3 * 60 + time_shifted.dt.hour * 60 + time_shifted.dt.minute

#     # Average out duplicate minutes
#     group = group.drop(['datetime', 'attraction', 'date'], axis = 1).groupby(['minute'])['SPOSTMIN'].mean().reset_index()

#     new_index = np.arange(0, 27 * 60 + 1, 30)  # Range from 0 to 1620 with a step of 30

#     # Step 2: Reindex the DataFrame
#     # Set the 'minute' column as the index
#     group = group.set_index('minute')

#     # Step 3: Reindex to the new index and interpolate
#     resampled = group.reindex(np.unique(np.concatenate([new_index, group.index]))).interpolate(method='linear')
#     resampled = resampled.loc[new_index]

#     # Step 4: Add zeros at the endpoints
#     #resampled.loc[0] = 0  # Set the first value to zero
#     #resampled.loc[1620] = 0  # Set the last value to zero

#     # Step 5: Reset index if needed
#     resampled = resampled.reset_index()

#     resampled['SPOSTMIN'] = resampled['SPOSTMIN'].fillna(0)
#     resampled['SPOSTMIN'] = (resampled['SPOSTMIN'] + 2.5) // 5 * 5
#     resampled.insert(0, 'date', date)
#     resampled.insert(0, 'attraction', attraction)
#     groups.append(resampled)

In [8]:
# df_netjes = pd.concat(groups, ignore_index=True)

In [9]:
# df_netjes.to_csv('dataset_Disney_clean_posted.csv')

In [10]:
# ## Actual waiting times only
# from tqdm import tqdm
# groups2 = list()
# for (date, attraction), group in tqdm(df_wait_act.groupby(['date', 'attraction'])):
# #for date, group in tqdm(list(df_wait_post.groupby('date'))):
#     time_shifted = group.datetime - datetime.timedelta(hours = 3)
#     group['minute'] = 3 * 60 + time_shifted.dt.hour * 60 + time_shifted.dt.minute

#     # Average out duplicate minutes
#     group = group.drop(['datetime', 'attraction', 'date'], axis = 1).groupby(['minute'])['SACTMIN'].mean().reset_index()

#     new_index = np.arange(0, 27 * 60 + 1, 30)  # Range from 0 to 1620 with a step of 30

#     # Step 2: Reindex the DataFrame
#     # Set the 'minute' column as the index
#     group = group.set_index('minute')

#     # Step 3: Reindex to the new index and interpolate
#     resampled = group.reindex(np.unique(np.concatenate([new_index, group.index]))).interpolate(method='linear')
#     resampled = resampled.loc[new_index]

#     # Step 4: Add zeros at the endpoints
#     #resampled.loc[0] = 0  # Set the first value to zero
#     #resampled.loc[1620] = 0  # Set the last value to zero

#     # Step 5: Reset index if needed
#     resampled = resampled.reset_index()

#     resampled['SACTMIN'] = resampled['SACTMIN'].fillna(0)
#     resampled['SACTMIN'] = (resampled['SACTMIN'] + 2.5) // 5 * 5
#     resampled.insert(0, 'date', date)
#     resampled.insert(0, 'attraction', attraction)
#     groups2.append(resampled)

In [11]:
# df_netjes_2 = pd.concat(groups, ignore_index=True)

In [12]:
# df_netjes_2.to_csv('dataset_Disney_clean_actuals.csv')

In [13]:
# ## Posted and actual waiting times

# groups3 = list()

# for (date, attraction), group in tqdm(df_wait.groupby(['date', 'attraction'])):
#     # Time shift by 3 hours
#     time_shifted = group['datetime'] - datetime.timedelta(hours=3)
#     group['minute'] = 3 * 60 + time_shifted.dt.hour * 60 + time_shifted.dt.minute

#     # Ensure the required columns exist before proceeding
#     if not {'SACTMIN', 'SPOSTMIN'}.issubset(group.columns):
#         print(f"Skipping {date} - {attraction}: Missing columns")
#         continue  # Skip this iteration if columns are missing

#     # Drop unnecessary columns and compute the mean for duplicate minutes
#     group = group.drop(columns=['datetime', 'attraction', 'date'], errors='ignore')  # Avoid KeyError
#     group = group.groupby('minute', as_index=False)[['SACTMIN', 'SPOSTMIN']].mean()

#     # Define the new index (0 to 1620 minutes in 30-minute intervals)
#     new_index = np.arange(0, 27 * 60 + 1, 30)

#     # Set 'minute' as the index and reindex with interpolation
#     group = group.set_index('minute')
#     reindexed = group.reindex(np.unique(np.concatenate([new_index, group.index]))).interpolate(method='linear')

#     # Keep only the required time indices
#     resampled = reindexed.loc[new_index].reset_index()

#     # Fill NaN values with 0 and round to nearest 5-minute interval
#     for col in ['SACTMIN', 'SPOSTMIN']:
#         if col in resampled.columns:
#             resampled[col] = resampled[col].fillna(0)
#             resampled[col] = ((resampled[col] + 2.5) // 5) * 5

#     # Insert date and attraction columns
#     resampled.insert(0, 'date', date)
#     resampled.insert(1, 'attraction', attraction)

#     # Append to final list
#     groups3.append(resampled)

In [14]:
# df_netjes_3 = pd.concat(groups3, ignore_index=True)

In [15]:
# df_netjes_3.to_csv('dataset_Disney_clean_all.csv')

<div style="background-color: rgba(0, 176, 240, 1); color: rgba(255, 255, 255, 1); font-size: 24px; font-weight: bold; padding: 10px; border-radius: 15px;">
    Interpolating and preparing training dataframe
</div>

In [16]:
groups_itp = list()

for (date, attraction), group in tqdm(df_wait.groupby(['date', 'attraction'])):
    # Time shift by 3 hours
    time_shifted = group['datetime'] - datetime.timedelta(hours=3)
    group['minute'] = 3 * 60 + time_shifted.dt.hour * 60 + time_shifted.dt.minute

    # Ensure the required columns exist before proceeding
    if not {'SACTMIN', 'SPOSTMIN'}.issubset(group.columns):
        print(f"Skipping {date} - {attraction}: Missing columns")
        continue  # Skip this iteration if columns are missing

    # Drop unnecessary columns and compute the mean for duplicate minutes
    group = group.drop(columns=['datetime', 'attraction', 'date'], errors='ignore')  # Avoid KeyError
    group = group.groupby('minute', as_index=False)[['SACTMIN', 'SPOSTMIN']].mean()

    # Define the new index (0 to 1620 minutes in 30-minute intervals)
    new_index = np.arange(0, 27 * 60 + 1, 30)

    # Set 'minute' as the index and reindex with interpolation
    group = group.set_index('minute')
    reindexed = group.reindex(np.unique(np.concatenate([new_index, group.index]))).interpolate(method='linear')

    # Keep only the required time indices
    resampled = reindexed.loc[new_index].reset_index()

    # Round to nearest 5-minute interval
    for col in ['SACTMIN', 'SPOSTMIN']:
        if col in resampled.columns:
            resampled[col] = ((resampled[col] + 2.5) // 5) * 5
    
    # Interpolate SPOSTMIN to fill blanks
    resampled['SPOSTMIN_interp'] = resampled['SPOSTMIN'].interpolate(method='linear')

    # Calculate the actual over posted ratio
    resampled['actual_over_posted'] =  resampled['SACTMIN'] / resampled['SPOSTMIN_interp']

    # Insert date and attraction columns
    resampled.insert(0, 'date', date)
    resampled.insert(1, 'attraction', attraction)

    # Append to final list
    groups_itp.append(resampled)

100%|██████████| 77543/77543 [13:55<00:00, 92.84it/s]  


In [19]:
df_netjes_itp = pd.concat(groups_itp, ignore_index=True)

In [20]:
df_netjes_itp

,date,attraction,minute,SACTMIN,SPOSTMIN,SPOSTMIN_interp,actual_over_posted
0,2015-01-01,7_dwarfs_train,0,NaN,NaN,NaN,NaN
1,2015-01-01,7_dwarfs_train,30,NaN,NaN,NaN,NaN
2,2015-01-01,7_dwarfs_train,60,NaN,NaN,NaN,NaN
3,2015-01-01,7_dwarfs_train,90,NaN,NaN,NaN,NaN
4,2015-01-01,7_dwarfs_train,120,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
4264860,2021-12-28,winnie_the_pooh,1500,15.0,5.0,5.0,3.0
4264861,2021-12-28,winnie_the_pooh,1530,15.0,5.0,5.0,3.0
4264862,2021-12-28,winnie_the_pooh,1560,15.0,5.0,5.0,3.0
4264863,2021-12-28,winnie_the_pooh,1590,15.0,5.0,5.0,3.0


In [21]:
# Save the full cleaned and interpolated dataset to csv
df_netjes_itp.to_csv("../data/clean/waiting_times_interpolated.csv")

In [22]:
# Select the dataset for training our prediction model and save it to csv
# Drop unnecessary columns and rows without actual_over_posted ratio
training_dataset = df_netjes_itp.drop(columns=['SACTMIN', 'SPOSTMIN', 'SPOSTMIN_interp']).dropna(subset=['actual_over_posted'])

training_dataset.to_csv("../data/clean/training_dataset.csv", index=False)